## GUI

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyodbc

# 🎨 Colors
BG_COLOR = "#f4f6f9"
PRIMARY_COLOR = "#283f57"
SECONDARY_COLOR = "#0f5786"
TEXT_COLOR = "#ffffff"

# ================= Database =================
def connect_db():
    try:
        conn = pyodbc.connect(
            "DRIVER={SQL Server};"
            "SERVER=NADA\\SQLEXPRESS;"        
            "DATABASE=HospitalDB;"   
            "Trusted_Connection=yes;"   
        )
        return conn
    except Exception as e:
        messagebox.showerror("Database Error", str(e))
        return None

# 📥 Load data
def load_data(table_name, tree_widget, filter_col=None, filter_val=None):
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        if filter_col and filter_val:
            query = f"SELECT * FROM {table_name} WHERE {filter_col} LIKE ?"
            cursor.execute(query, ('%' + filter_val + '%',))
        else:
            query = f"SELECT * FROM {table_name}"
            cursor.execute(query)

        rows = cursor.fetchall()
        cols = [column[0] for column in cursor.description]

        # Clear old data
        for col in tree_widget.get_children():
            tree_widget.delete(col)

        # Update columns
        tree_widget["columns"] = cols
        tree_widget["show"] = "headings"
        for col in cols:
            tree_widget.heading(col, text=col)
            tree_widget.column(col, width=120, anchor="center")

        # Insert new rows
        for row in rows:
            clean_row = [str(v).replace(";", "").replace('"', '') for v in row]
            tree_widget.insert("", "end", values=clean_row)

        
        if tree_widget == tree:  
            col_combo["values"] = cols
            if cols:
                col_combo.current(0) 

        conn.close()
        return cols

# 📤 Insert / Update / Delete
def add_row(table, entries, tree_widget):
    values = [entry.get() for entry in entries]
    if not all(values):
        messagebox.showwarning("Validation", "All fields must be filled")
        return
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        placeholders = ",".join("?" * len(values))
        query = f"INSERT INTO {table} VALUES ({placeholders})"
        try:
            cursor.execute(query, values)
            conn.commit()
            messagebox.showinfo("Success", "Row added successfully")
            load_data(table, tree_widget)
        except Exception as e:
            messagebox.showerror("Insert Error", str(e))
        conn.close()

def update_row(table, entries, tree_widget):
    selected = tree_widget.selection()
    if not selected:
        messagebox.showwarning("Select", "Select a row to update.")
        return
    values = [entry.get() for entry in entries]
    if not all(values):
        messagebox.showwarning("Validation", "All fields must be filled")
        return
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cols = [tree_widget.heading(c)["text"] for c in tree_widget["columns"]]
        set_clause = ",".join([f"{col}=?" for col in cols[1:]])  # skip PK
        query = f"UPDATE {table} SET {set_clause} WHERE {cols[0]}=?"
        try:
            cursor.execute(query, values[1:] + [values[0]])
            conn.commit()
            messagebox.showinfo("Success", "Row updated successfully")
            load_data(table, tree_widget)
        except Exception as e:
            messagebox.showerror("Update Error", str(e))
        conn.close()

def delete_row(table, tree_widget):
    selected = tree_widget.selection()
    if not selected:
        messagebox.showwarning("Select", "Select a row to delete.")
        return
    row = tree_widget.item(selected[0])["values"]
    conn = connect_db()
    if conn:
        cursor = conn.cursor()
        cols = [tree_widget.heading(c)["text"] for c in tree_widget["columns"]]
        pk = cols[0]
        query = f"DELETE FROM {table} WHERE {pk}=?"
        try:
            cursor.execute(query, row[0])
            conn.commit()
            messagebox.showinfo("Success", "Row deleted successfully")
            load_data(table, tree_widget)
        except Exception as e:
            messagebox.showerror("Delete Error", str(e))
        conn.close()

# --- GUI Setup ---
root = tk.Tk()
root.title("🏥 Hospital Database Management System")
root.geometry("1200x700")
root.configure(bg=BG_COLOR)

# Tabs
notebook = ttk.Notebook(root)
notebook.pack(fill="both", expand=True, padx=10, pady=10)

# 📊 Tab: View & Filter
view_tab = tk.Frame(notebook, bg=BG_COLOR)
notebook.add(view_tab, text="📊 View & Filter Data")

title_label = tk.Label(view_tab, text="Hospital Database Viewer",
                       bg=PRIMARY_COLOR, fg=TEXT_COLOR,
                       font=("Arial", 20, "bold"), pady=10)
title_label.pack(fill="x")

table_frame = tk.Frame(view_tab, bg=BG_COLOR)
table_frame.pack(pady=10)

tk.Label(table_frame, text="Select Table:", bg=BG_COLOR,
         fg=PRIMARY_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=0, padx=5)

tables = ["Department", "Doctor", "Nurse", "Patient", "Staff",
          "Room", "Medical_Record", "Ambulance", "Emergency_Center", "Medical_Drugs"]

table_combo = ttk.Combobox(table_frame, values=tables, font=("Arial", 12), width=18)
table_combo.grid(row=0, column=1, padx=5)

refresh_btn = tk.Button(table_frame, text="Refresh Data",
                        command=lambda: load_data(table_combo.get(), tree),
                        bg=SECONDARY_COLOR, fg=TEXT_COLOR,
                        font=("Arial", 12, "bold"))
refresh_btn.grid(row=0, column=2, padx=10)

# Filter Section
filter_frame = tk.Frame(view_tab, bg=BG_COLOR)
filter_frame.pack(pady=10)

tk.Label(filter_frame, text="From:", bg=BG_COLOR,
         fg=PRIMARY_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=0, padx=5)

col_combo = ttk.Combobox(filter_frame, font=("Arial", 12), width=18)
col_combo.grid(row=0, column=1, padx=5)

tk.Label(filter_frame, text="Value:", bg=BG_COLOR,
         fg=PRIMARY_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=2, padx=5)

filter_entry = tk.Entry(filter_frame, font=("Arial", 12), width=20)
filter_entry.grid(row=0, column=3, padx=5)

filter_btn = tk.Button(filter_frame, text="Filter",
                       command=lambda: load_data(table_combo.get(), tree, col_combo.get(), filter_entry.get()),
                       bg="#27ae60", fg=TEXT_COLOR, font=("Arial", 12, "bold"))
filter_btn.grid(row=0, column=4, padx=5)

clear_btn = tk.Button(filter_frame, text="Clear",
                      command=lambda: load_data(table_combo.get(), tree),
                      bg="#e74c3c", fg=TEXT_COLOR, font=("Arial", 12, "bold"))
clear_btn.grid(row=0, column=5, padx=5)

tree = ttk.Treeview(view_tab)
tree.pack(expand=True, fill="both", padx=20, pady=10)

# 📋 Tab: Manage Data (CRUD)
crud_tab = tk.Frame(notebook, bg=BG_COLOR)
notebook.add(crud_tab, text="🛠 Manage Data (CRUD)")

crud_title = tk.Label(crud_tab, text="Manage Records",
                      bg=PRIMARY_COLOR, fg=TEXT_COLOR,
                      font=("Arial", 20, "bold"), pady=10)
crud_title.pack(fill="x")

crud_top = tk.Frame(crud_tab, bg=BG_COLOR)
crud_top.pack(pady=10)

tk.Label(crud_top, text="Select Table:", bg=BG_COLOR,
         fg=PRIMARY_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=0, padx=5)

crud_table_combo = ttk.Combobox(crud_top, values=tables, font=("Arial", 12), width=18)
crud_table_combo.grid(row=0, column=1, padx=5)

crud_tree = ttk.Treeview(crud_tab)
crud_tree.pack(expand=True, fill="both", padx=20, pady=10)

form_frame = tk.Frame(crud_tab, bg=BG_COLOR)
form_frame.pack(fill="x", padx=20, pady=10)

entry_widgets = []

def load_crud_table():
    table = crud_table_combo.get()
    if table:
        cols = load_data(table, crud_tree)
        for widget in entry_widgets:
            widget.destroy()
        entry_widgets.clear()
        for idx, col in enumerate(cols):
            tk.Label(form_frame, text=col, bg=BG_COLOR, fg=PRIMARY_COLOR,
                     font=("Arial", 10, "bold")).grid(row=idx, column=0, padx=5, pady=2)
            entry = tk.Entry(form_frame, font=("Arial", 10), width=25)
            entry.grid(row=idx, column=1, padx=5, pady=2)
            entry_widgets.append(entry)

tk.Button(crud_top, text="Load Table", command=load_crud_table,
          bg=SECONDARY_COLOR, fg=TEXT_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=2, padx=10)

# Buttons
btns_frame = tk.Frame(crud_tab, bg=BG_COLOR)
btns_frame.pack(pady=10)

tk.Button(btns_frame, text="Add", command=lambda: add_row(crud_table_combo.get(), entry_widgets, crud_tree),
          bg=SECONDARY_COLOR, fg=TEXT_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=0, padx=5)

tk.Button(btns_frame, text="Update", command=lambda: update_row(crud_table_combo.get(), entry_widgets, crud_tree),
          bg="#27ae60", fg=TEXT_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=1, padx=5)

tk.Button(btns_frame, text="Delete", command=lambda: delete_row(crud_table_combo.get(), crud_tree),
          bg="#e74c3c", fg=TEXT_COLOR, font=("Arial", 12, "bold")).grid(row=0, column=2, padx=5)

root.mainloop()
